# TTS Deploy on embedded

In this tutorial we will explain the process of generating TTS RMIR from synthesizer.riva and vocoder.riva files for embedded machines. RMIR (Riva Model Intermediate Representation) is an intermediate file that has all the necessary artifacts (models, files, configurations, and user settings) required to deploy a Riva service.  

This tutorial assumes that the .riva files are present for both synthesizer and vocoder. [`nemo2riva`](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/model-overview.html#export-models-with-nemo2riva) can be used to generate .riva files from nemo checkpoints.  

The RMIR generated in this tutorial can be deployed using [riva_quickstart](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/quick-start-guide.html)

import pathlib to get the paths

In [ ]:
import pathlib

### Download models
We will download pretrained [Fastpitch](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/speechsynthesis_en_us_fastpitch_ipa) and [HifiGan](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/speechsynthesis_en_us_hifigan_ipa) models from ngc. You can replace these models with the paths of your custom model, incase of custom models.

In [ ]:
!ngc registry model download-version "nvidia/tao/speechsynthesis_en_us_fastpitch_ipa:deployable_v1.0"
!ngc registry model download-version "nvidia/tao/speechsynthesis_en_us_hifigan_ipa:deployable_v1.0"

### Set configs and params.
Set following config parameters:  
`synthesizer`: Full path for synthesizer.riva file from [ngc](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/speechsynthesis_en_us_fastpitch_ipa)  
`vocoder`: Full path for vocoder.riva file file from [ngc](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/speechsynthesis_en_us_hifigan_ipa)  
`out_dir`: Directory to put TTS.rmir file  
`voice`: Set the voice name of the model.  
`key`: This is the encryption key used in nemo2riva. Same key will be used to deploy the RMIR generated in this tutorial.  
`use_ipa`: Set to "y" or "Y" if the model uses IPA phones, "no" if the model uses arpabet.  
`lang`: Model language.  
`sample_rate`: Sample rate of generated audios.  

In [ ]:
synthesizer = pathlib.Path.cwd() / "speechsynthesis_en_us_fastpitch_ipa_vdeployable_v1.0/FastPitch_44k_EnglishUS_IPA.riva" ##Synthesizer .riva location
vocoder = pathlib.Path.cwd() / "speechsynthesis_en_us_hifigan_ipa_vdeployable_v1.0/HifiGAN_44k_EnglishUS_IPA.riva" ##vocoder .riva location
out_dir = pathlib.Path("out/") ##Output directory to store generated RMIR.
voice = "test" ##Voice name
key = "tlt_encode" ##Encryption key used during nemo2riva
use_ipa = "no" ##"y" or "Y" if the model uses ipa, no otherwise.
lang = "en-US" ##Language
sample_rate = 44100 ##Sample rate of the audios


## Riva NGC, servicemaker image config.
riva_ngc_org = "nvidia"
riva_ngc_team = "riva"
NGC_TARGET = f"{riva_ngc_org}/{riva_ngc_team}"
riva_ngc_image_version = "2.8.0"
riva_init_image = f"nvcr.io/{NGC_TARGET}/riva-speech:{riva_ngc_image_version}-servicemaker-l4t-aarch64"

### Download riva model repository files from ngc

In [ ]:
!ngc registry model download-version "nvidia/tao/speechsynthesis_en_us_auxiliary_files:deployable_v1.3"

Set the riva_model_repo path

In [ ]:
riva_model_files=pathlib.Path.cwd() / "speechsynthesis_en_us_auxiliary_files_vdeployable_v1.3"

Get synthesizer, vocoder directory path and model names.

In [ ]:
synt_dir = synthesizer.parent
voc_dir = vocoder.parent

synt_name = synthesizer.name
voc_name = vocoder.name

Stop already running docker file and run riva_servicemaker and run again with synthesizer and vocoder paths.

In [ ]:
##Run the riva servicemaker.
!docker stop riva_rmir_gen &> /dev/null
!set -x && docker run -td --gpus all --rm -v {str(riva_model_files)}:/riva_repo -v {str(synt_dir)}/:/synt -v {str(voc_dir)}:/voc \
            -v {str(out_dir)}:/data --name riva_rmir_gen --entrypoint="/bin/bash" {riva_init_image}

In [ ]:
riva_build=f"""riva-build speech_synthesis --force  --voice_name={voice}  --language_code={lang} \
                --sample_rate={sample_rate} /data/TTS.rmir:{key} /synt/{synt_name}:{key} \
                /voc/{voc_name}:{key} --max_batch_size 1 --denoiser.max_batch_size 1 --preprocessor.max_batch_size 1 \
                --encoderFastPitch.max_batch_size 1 --chunkerFastPitch.max_batch_size 1 --hifigan.max_batch_size 1 \
                --abbreviations_file=/riva_repo/abbr.txt"""

In [ ]:
if use_ipa == "Y" or use_ipa=="y":
    riva_build+=" --arpabet_file=/riva_repo/ipa_cmudict-0.7b_nv22.08.txt"
else:
    riva_build+=" --arpabet_file=/riva_repo/cmudict-0.7b_nv22.08"
print(riva_build)

Execute the riva build command and stop the riva_servicemaker container.

In [ ]:
!docker exec  riva_rmir_gen {riva_build}
!docker stop riva_rmir_gen

## Conclusion.

In this tutorial, we learned how to generate RMIR files from .riva files. We would see that a `TTS.rmir` has been generated in the `out_dir` location we defined earlier.  

The RMIR file generated in this tutorial can be deployed using [riva_quickstart](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/quick-start-guide.html)